# 排座位程式
這個程式可以根據提供的學生列表和座位表尺寸，生成座位表並將結果保存為 HTML 。以下是使用這個程式的步驟：
1. 準備學生列表：利用數位學苑 2.0 或 3.0，將學生的名單保存在一個檔案中，並在匯出名單時，記得取消選取自己的名字。若是數位學苑 2.0，使用課程功能旁邊的成員選項；若是數位學苑 3.0，匯出的檔案類型選擇 csv。

2. 決定座位表尺寸：決定座位表的行數和列數。可以根據不同教室數量生成多個座位表。 

3. 決定其他選項：選擇是否安排梅花座 (floral_arrangement)，以及是否隨機打亂學生列表 (shuffle)。

4. 指定輸出 HTML 文件的路徑：輸入要生成的 HTML 文件的路徑。

5. 執行程式：將上述資訊作為參數，執行 `create_seating_charts` 函式。

6. 檢查生成的 HTML 文件：執行程式後，可使用預設瀏覽器打開生成的 HTML 。在瀏覽器中檢查座位表，確保生成結果符合預期。

這就是使用座位表生成程式的基本步驟。請按照上述指南操作，根據您的需求生成座位表並享受方便快捷的座位安排。

## 參數
- `path`：包含學生列表的檔案路徑。

- `html_path`：生成的 HTML 的路徑。

- `num_rows_list`：各個教室座位表的列數列表。 ex : `[10]` 代表一間教室 10 列，`[11, 12]` 代表兩間教室，第一間 11 列，第二間 12 列。

- `num_cols_list`：各個教室座位表的行數列表。

- `html_titles`：每個座位表的所在教室。 ex : `['2F07', '2F04']`。

- `two`（可選）：是否為數位學苑 2.0，預設為 `False`。

- `floral_arrangement`（可選）：是否安排梅花座，預設為 `False`。

- `shuffle`（可選）：是否隨機打亂學生列表，預設為 `False`。

In [11]:
def split_dataframe(df, ratios):
    total_ratio = sum(ratios)  # 計算比例的總和
    normalized_ratios = [ratio / total_ratio for ratio in ratios]  # 計算標準化後的比例

    # 切割 DataFrame
    subsets = []
    start = 0
    for ratio in normalized_ratios:
        end = start + int(ratio * len(df)) + 1
        subset = df.iloc[start:end]
        subsets.append(subset)
        start = end
    
    return subsets


In [12]:
import pandas as pd
import numpy as np

def create_seating_chart(df, num_rows, num_cols, two = False, floral_arrangement = False, shuffle = False):
    if shuffle:
        df = df.sample(frac = 1).reset_index(drop = True) # 若需打亂學生列表，則使用 pandas 的 sample 函數重新排序並重置索引
        
    seating_chart = pd.DataFrame('', index = range(num_rows), columns = range(num_cols))  # 建立座位表，預設為空白

    if floral_arrangement:
        seating_chart = create_floral_arrangement(seating_chart, num_rows, num_cols)

    filled_seats = 0  # 計算已安排座位的人數
    for row in range(num_rows):
        for col in range(num_cols):
            if seating_chart.iloc[row, col] == '':
                if filled_seats < len(df):  # 若座位表仍有空位且學生尚未排完
                    member = df.iloc[filled_seats]
                    if two == True:
                        seating_chart.iloc[row, col] = f"{member[1]} ({member[0]}) - {member[3]}"
                    else:
                        seating_chart.iloc[row, col] = f"{member[0]} ({member[1]}) - {member[3]}"  # 在座位表中填入學生資訊
                    filled_seats += 1  # 已安排座位的人數加 1

    if filled_seats < len(df):  # 若座位表已填入的人數仍小於學生人數
        remaining_members = df.iloc[filled_seats:]
        available_positions = np.where(seating_chart.values == 'O')  # 尋找剩餘的空位位置

        if len(available_positions[0]) > 0:  # 若還有空位
            for i, member in enumerate(remaining_members.values):
                if i < len(available_positions[0]):
                    row, col = available_positions[0][i], available_positions[1][i]
                    if two == True:
                        seating_chart.iloc[row, col] = f"{member[1]} ({member[0]}) - {member[3]}"
                    seating_chart.iloc[row, col] = f"{member[0]} ({member[1]}) - {member[3]}"

    return seating_chart

def create_floral_arrangement(seating_chart, num_rows, num_cols):
    seats = seating_chart.values
    seats[seats == ''] = 'O'  # 將空位標記為 'O'

    for row in range(num_rows):
        for col in range(num_cols):
            if (row % 2 == 0 and col % 2 == 0) or (row % 2 == 1 and col % 2 == 1):
                if seats[row, col] == 'O':
                    if row - 1 >= 0:
                        seats[row - 1, col] = 'X'
                    if row + 1 < num_rows:
                        seats[row + 1, col] = 'X'
                    if col - 1 >= 0:
                        seats[row, col - 1] = 'X'
                    if col + 1 < num_cols:
                        seats[row, col + 1] = 'X'

    seating_chart = pd.DataFrame(seats, index = range(num_rows), columns = range(num_cols))
    seating_chart.replace({'O': ''}, inplace = True)
    seating_chart.replace({'X': 'O'}, inplace = True)

    return seating_chart


In [13]:
def create_seating_charts(path, html_path, num_rows_list, num_cols_list, html_titles, two = False, floral_arrangement = False, shuffle = False):
    if two == True:
        members = pd.read_excel(path)
        members = members.drop(members.index[0])
    else:
        members = pd.read_csv(path)  # 讀取 CSV 檔案
    full_list = np.array(num_rows_list) * np.array(num_cols_list)  # 計算每個教室的總座位數量
    member_list = split_dataframe(members, full_list)  # 分配學生至不同教室
    html = ""

    # 針對每個教室生成座位表
    for i, member_subset in enumerate(member_list):
        seating_chart = create_seating_chart(member_subset, num_rows_list[i], num_cols_list[i], two = two, floral_arrangement = floral_arrangement, 
                                             shuffle = shuffle)
        seating_chart = seating_chart.reset_index(drop = True)
        seating_chart.columns = seating_chart.columns + 1
        seating_chart.index = seating_chart.index + 1
        
        # 生成 HTML 標題
        html += f"<h1 style='text-align: center;'>{html_titles[i]}</h1>"
        html += "<h2 style='text-align: center;'>講台</h2>"

        html += seating_chart.to_html()  # 將座位表轉換成 HTML 格式並添加到 HTML 字串中

        # 如果不是最後一個教室，則添加分頁符號
        if i < len(member_list) - 1:
            html += "<div style='page-break-after: always;'></div>"  

    # 將 HTML 字串寫入 HTML 檔案
    with open(html_path, 'w', encoding = 'cp950') as f:
        f.write(html)


In [14]:
create_seating_charts('/Users/loyichun/Downloads/courseid_6295_participants.csv',
                      '/Users/loyichun/Desktop/seating_chart.html',
                      [11, 6], [10, 8], ['2F07', '2F04'], False, True, True)


/var/folders/lq/18q58sb57vg0s_4kf5q9bzvm0000gn/T/ipykernel_24235/2624446143.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seating_chart.iloc[row, col] = f"{member[0]} ({member[1]}) - {member[3]}"  # 在座位表中填入學生資訊
